# Calculating XY Tool offsets Helper Notebook

In [1]:
import sys
sys.path.append('..')
from utils.MachineUtils import *
from utils.CameraUtils import *
import cv2 as cv2

In [2]:
# Setup your machine connection
# List available ports in thie cell

ports = serial.tools.list_ports.comports()
print([port.name for port in ports]) 

['ttyACM0', 'ttyAMA0']


In [9]:
# Choose the correct port from above and establish connection with machine
port = '/dev/ttyACM0'
m = MachineCommunication(port)

In [ ]:
# Make sure no XY tool offsets are set for your second tool
# TODO: send G10 commands from jupyter
# for now, send from duet console

In [ ]:
# Pick up the tool and move it to a reference position (e.g. 150, 150)
tool_number = 4
m.toolChange(tool_number)
m.moveTo(x=59.9, y=131.6) # approx location on magnetic bed


In [ ]:
# Move the bed down and install the USB microscope onto the bed facing up
m.move_to(z=150)

In [18]:
# Open a live video window and center the z-probe
# choose the correct video device if you have >1 camera
cap = cv2.VideoCapture(0) #Note that the index corresponding to your camera may not be zero but this is the most common default

# draw a circle in the center of the frame
center = None
while center is None:
    # the first frame grab is sometimes empty
    ret, frame = cap.read()
    h, w = frame.shape[0:2]
    print(h)
    print(w)
    center = (int(w/2), int(h/2))
    print(center)

while True:
    ret, frame = cap.read()
    target = cv2.circle(frame, center, 5, (0,255,0), -1)
    cv2.imshow('Input', frame)
    c = cv2.waitKey(1)
    if c ==27: #27 is the built in code for ESC so press escape to close the window. 
        break 
        
cap.release()
cv2.destroyAllWindows()

480
640
(320, 240)


In [19]:
# record the coordinates at which the z-probe is centered
pos = m.get_position()
print(pos)
zprobe_x = float(pos['X'])
zprobe_y = float(pos['Y'])
print(zprobe_x, zprobe_y) # using a height of 150 with tool equipped

{'X': '149.900', 'Y': '150.000', 'Z': '130.000', 'U': '0.000', 'V': '0.500', 'E': '0.000', 'E0': '-0.0'}
149.9 150.0


In [20]:
# open a live feed again, and center the equipped tool

cap = cv2.VideoCapture(0) #Note that the index corresponding to your camera may not be zero but this is the most common default

# draw a circle in the center of the frame
center = None
while center is None:
    # the first frame grab is sometimes empty
    ret, frame = cap.read()
    h, w = frame.shape[0:2]
    print(h)
    print(w)
    center = (int(w/2), int(h/2))
    print(center)

while True:
    ret, frame = cap.read()
    target = cv2.circle(frame, center, 5, (0,255,0), -1)
    cv2.imshow('Input', frame)
    c = cv2.waitKey(1)
    if c ==27: #27 is the built in code for ESC so press escape to close the window. 
        break 
        
cap.release()
cv2.destroyAllWindows()

480
640
(320, 240)


In [21]:
# record these coordinates
pos = m.get_position()
tool_x = float(pos['X'])
tool_y = float(pos['Y'])
print(tool_x, tool_y)

150.6 106.2


In [22]:
# subtract your tool (x,y) coords from the probe (x,y) coords - this is your XY tool offset
# add the following command to your Toffsets.g file
xoff = zprobe_x - tool_x
yoff = zprobe_y - tool_y
tool_number = 1
print(f"G10 P{tool_number} X{xoff:.2f} Y{yoff:.2f}")

# BUT I have X-4.5 Y44.04 in TOffsets.g for p2 already. so... add them to this gets me 
# the right Toffset
#e.g. if the center of syringe was 152.4, 154.5, then i apply -4.5-2.4 = -6.9, 44.04 - 4.5 = 39.54
# --> G10 P2 X-6.9 Y39.54
# Apply directly in GCode
# TODO: add G10 commands from jupyter
# for now, send from duet console
# G10 P1 X0.5 Y

G10 P1 X-0.70 Y43.80


In [ ]:
# once the tool is centered, take a pic of the first tool
# (close vlc/any video player so we have access to the camera here)
%matplotlib inline
tool1 = getFrameCamera(0)
showFrame(tool1, grid=True)
cv2.imwrite("/home/pi/zprobe.jpg", tool1)

In [ ]:
# now pick up the second tool & move it to the reference point set by first tool
m.toolChange(0)
m.moveTo(x=64.3, y=147.7)

In [ ]:
#Open a live video window
# choose the correct video device if you have >1 camera
# center the tool under the microscope
# make note of the x,y coordinates
cap = cv2.VideoCapture(0) #Note that the index corresponding to your camera may not be zero but this is the most common default

# draw a circle in the center of the frame
center = None
while center is None:
    # the first frame grab is sometimes empty
    ret, frame = cap.read()
    h, w = frame.shape[0:2]
    print(h)
    center = (int(w/2), int(h/2))

while True:
    ret, frame = cap.read()
    target = cv2.circle(frame, center, 5, (0,255,0), -1)
    cv2.imshow('Input', frame)
    c = cv2.waitKey(1)
    if c ==27: #27 is the built in code for ESC so press escape to close the window. 
        break 
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
# take a picture of the second tool
tool2 = getFrameCamera(1)
showFrame(tool2, grid=True)
cv2.imwrite("/home/pi/tool2.jpg", tool2)

In [ ]:
# compare the two pictures above
# decide which direction the second tool needs to move to be centered
# m.moveTo(x=150, y=150) # change based on your picture

tool2 = getFrameCamera(1)
showFrame(tool2, grid=True)

In [ ]:
# subtract your final values above from 150 - this is your XY tool offset
# BUT I have X-4.5 Y44.04 in TOffsets.g for p2 already. so... add them to this gets me 
# the right Toffset
#e.g. if the center of syringe was 152.4, 154.5, then i apply -4.5-2.4 = -6.9, 44.04 - 4.5 = 39.54
# --> G10 P2 X-6.9 Y39.54
# Apply directly in GCode
# TODO: add G10 commands from jupyter
# for now, send from duet console

In [ ]:
# Now move to the reference point
# the tool should be centered (i.e. not move)
m.moveTo(x=57.9, y=153.4)

In [ ]:
# # Do a toolchange from scratch to make sure
#m.toolChange(1)
# m.toolChange(2)
#m.moveTo(x=73, y=144.7)
toolCheck = getFrameCamera(1)
showFrame(toolCheck, grid=True)